In [2]:
import csv
import numpy as np

In [3]:
with open('../DADOS/MICRODADOS_ENEM_2018.csv', errors='ignore') as dataCSV:
    data_csv = csv.reader(dataCSV, delimiter=',')
    data_list_csv = list(data_csv)


In [3]:
print(len(data_list_csv))

5513748


In [4]:
# Get the answers vector and the math grades 
print("oi")
y = []
x = []
i = 0

#Age field - used as reference for location of different fields (the next is ref_field + 1 and so on...)
ref_field = 59                          
x_social =  []

for row in data_list_csv:
    parameters = row[0].split(';')
    print(parameters[85])
    print(parameters)
    if(parameters[85] == '1'):
        #print("oi")
        if(parameters[102][0] == 'E'):        
            nota_mt = float(parameters[93])
            y.append(nota_mt)
            answers = [int(ans == gab) for (ans, gab) in zip(parameters[97], parameters[102])]
            x.append(answers)
            #x_social.append([parameters[2],  parameters[13], parameters[15], parameters[18], parameters[27]])
    break
    
    

np.save("socialData.npy", x_social)
np.save("answers.npy", x)
np.save("grades.npy", y)

print(len(y))

oi
TP_PRESENCA_MT
['NU_INSCRICAO', 'NU_ANO', 'CO_MUNICIPIO_RESIDENCIA', 'NO_MUNICIPIO_RESIDENCIA', 'CO_UF_RESIDENCIA', 'SG_UF_RESIDENCIA', 'NU_IDADE', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'CO_MUNICIPIO_NASCIMENTO', 'NO_MUNICIPIO_NASCIMENTO', 'CO_UF_NASCIMENTO', 'SG_UF_NASCIMENTO', 'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO', 'IN_TREINEIRO', 'CO_ESCOLA', 'CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC', 'CO_UF_ESC', 'SG_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC', 'IN_BAIXA_VISAO', 'IN_CEGUEIRA', 'IN_SURDEZ', 'IN_DEFICIENCIA_AUDITIVA', 'IN_SURDO_CEGUEIRA', 'IN_DEFICIENCIA_FISICA', 'IN_DEFICIENCIA_MENTAL', 'IN_DEFICIT_ATENCAO', 'IN_DISLEXIA', 'IN_DISCALCULIA', 'IN_AUTISMO', 'IN_VISAO_MONOCULAR', 'IN_OUTRA_DEF', 'IN_GESTANTE', 'IN_LACTANTE', 'IN_IDOSO', 'IN_ESTUDA_CLASSE_HOSPITALAR', 'IN_SEM_RECURSO', 'IN_BRAILLE', 'IN_AMPLIADA_24', 'IN_AMPLIADA_18', 'IN_LEDOR', 'IN_ACESSO', 'IN_TRANSCRICAO', 'IN_LIBRAS', 'IN_LEITURA_LABI

In [ ]:
# Faster load

y = np.load("grades.npy")
x = np.load("answers.npy")
x_social = np.load("socialData.npy")


In [5]:
# Divide in train and test

x_train = x[:(int) (0.8 * len(x))]
x_test = x[(int) (0.8*len(x)):]
y_train = y[:(int) (0.8 * len(y))]
y_test = y[(int) (0.8*len(y)):]

In [6]:
# Distribuicao de todas as notas

import matplotlib.pyplot as plt

print(np.mean(y))
plt.hist(y, rwidth=0.9)
plt.show()

536.0135401876072


<Figure size 640x480 with 1 Axes>

In [8]:
#treinando regressao linear basica a partir das respostas

import sklearn
from sklearn.linear_model import Lasso
import numpy as np

alpha = 0

max_score = False

model = Lasso(alpha, warm_start=True, max_iter=1000).fit(x_train, y_train)
model_score = model.score(x_test, y_test)
print("oi ", np.linalg.norm(y_test - model.predict(x_test)) / len(y_test)**(1/2))
num_questions = np.count_nonzero(model.coef_)
print("Score %f with %d questions" %(model_score, num_questions))
if(model_score > max_score or (not max_score)):
    max_score = model_score
    model_min = model        
    max_alpha = alpha
    print("Best Score changed to %f" %(max_score))
    
print("Best Score is to %f with alpha %f" %(max_score, max_alpha))
print(model_min.coef_)



/usr/lib/python3/dist-packages/ipykernel_launcher.py:11: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  # This is added back by InteractiveShellApp.init_path()
/home/carlosf/.local/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
/home/carlosf/.local/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 139401733.61583057, tolerance: 842610.0814850277
  positive)


oi  18.89600356885822
Score 0.964521 with 44 questions
Best Score changed to 0.964521
Best Score is to 0.964521 with alpha 0.000000
[18.97019959  6.35954363 48.53396062 15.04744793 29.03509794 39.95425624
 23.83866366  2.54553826  9.56779036  9.22518857  8.20740732  7.33013022
  3.13717365  3.83870597 17.70083968  1.64800897 26.40648939  3.08434604
  9.31777853  2.29153679 11.45883976  0.52257583 -0.0814632   5.9957213
  1.1562043  15.68676076 23.90054966 31.83758865  8.85633977 22.68656424
  0.79731712 14.28427113  5.74149844 61.19571243 10.52526991  6.04389548
 10.38039397 18.3100156   2.81102608 15.84455394 13.45864103 12.79625535
  4.30681002 19.55715197  0.        ]


In [ ]:
import matplotlib.pyplot as plt

historic_values = []

K = 4 * 10e-4


rating_questions = avg_grades * np.ones(45)
for i in range(1, len(y_train)):
    rating_questions = match(K, rating_questions, y_train[i], 1 - x_train[i])
historic_values.append(rating_questions[0])
    
print(rating_questions)



In [ ]:
plt.hist(rating_questions)
plt.show()

In [ ]:
K_range = [30]



for K in K_range:
    print(K)
    notes_test = avg_grades * np.ones(len(y_test))
    for j in range(1, len(y_test)):
        for i in range(1, 45):
            notes_test[j] = match(K, notes_test[j], rating_questions[i], x_test[j][i])
    print(np.linalg.norm(notes_test - y_test) / len(y_test) ** (1/2))
    
    

In [ ]:
def match(K, ratingQuestion, ratingStudent, resultMatch):
    return ratingQuestion + K * (resultMatch - 1 / (1 + 10**((ratingStudent - ratingQuestion)/400)) )